# Relay

Graphene has complete support for [Relay](https://relay.dev/docs/guides/graphql-server-specification/) and offers some utils to make integration from Python easy.


**Summary**

- Use `Node` to execute query, include query `Typed Node` and query `Typeless Node`
- Use `CustomNode`:
    - Define custom *global id*
    - Define custom *type resolve* method
    - Use `possible_types` meta define
- Define `Connection`:
    - Use `ArrayConnection` to query list field
    - Define *connection field resolve*, *page info resolve*, *edges resolve* and so on
- Define `Mutation`:
    - Use `ClientIDMutation` to define Mutation simply

In [ ]:
import json


def pr(r):
    if r.errors:
        errors = [
            {
                "message": e.message, "locations": [
                    {
                        "line": lo.line, "column": lo.column
                    } for lo in e.locations
                ] if e.locations else []
            } for e in r.errors]
        s = json.dumps(errors, indent=4).replace("\n", "\n    ")
        return f"Error: {s}"

    s = json.dumps(r.data, indent=4).replace("\n", "\n    ")
    return f"Data: {s}"

## 1. Nodes

A `Node` is an Interface provided by `graphene.relay` that contains a single field `id` (which is a `ID!`). Any object that inherits from it has to implement a `get_node` method for retrieving a `Node` by an *id*.

Note: 
- `Node`是一个`Interface`类型的子类，作用是使用一个**编码过**的ID，通过`get_node`方法获取对象
- 使用`Node`可以获得包含"类型:ID"信息的数据，从而完成相关数据的获取，而无需在`root`类型上使用`resolve`方法

### 1.1. Quick example (See also [Starwars Relay example](https://github.com/graphql-python/graphene/blob/master/examples/starwars_relay/schema.py))

Example usage:

- Define dataset

In [ ]:
from promise import Promise
from promise.dataloader import DataLoader


class Dataset:
    def __init__(self):
        self.ships = {}

    def get_ship(self, id):
        return self.ships[id]

    def save_ship(self, ship):
        self.ships[ship.id] = ship


dataset = Dataset()


class ShipLoader(DataLoader):
    def batch_load_fn(self, keys):
        ships = map(lambda key: dataset.get_ship(key), keys)
        return Promise.resolve(list(ships))


ship_loader = ShipLoader()

- Define ObjectType and Schema

In [ ]:
from graphene import ObjectType, String, ID, Field, Argument, Schema
from graphene.relay import Node


class Ship(ObjectType):
    """ A ship in the Star Wars saga """
    class Meta:
        interfaces = (Node,)

    name = String(required=True)

    @classmethod
    def get_node(cls, info, id):
        return ship_loader.load(int(id))  # Must return "ObjectType" object


class Query(ObjectType):
    ship = Field(Ship, id=Argument(ID))
    ship_node = Node.Field(Ship)
    node = Node.Field()

    @staticmethod
    def resolve_ship(root, info, id):
        return ship_loader.load(int(id))


schema = Schema(query=Query)

Then query contains three field: `ship`(normal `Scalar` field), `shipNode`(Typed `Node` field) and `node`(Typeless `Node` field)

- Fill data to dataset

In [ ]:
for id in range(1, 51):
    dataset.save_ship(
        Ship(
            id=id,
            name=f"Ship-{id}"
        )
    )

- Query "ship" field

In [ ]:
import base64


q = """
    query getShipById($id: ID!) {
        ship(id: $id) {
            id
            name
        }
    }"""
v = {"id": 10}
r = schema.execute(q, variables=v)
print(f'* The "r.data" of query "{q}, variables={v}\n" is: "{pr(r)}"')

id = r.data["ship"]["id"]
decode_id = base64.b64decode(id.encode()).decode()
print(f'  and the decoded value of "r.data["ship"]["id"]" is: "{decode_id}"')

The `id` returned by the `Ship` type when you query it will be a *scalar* which contains enough info for the server to know its type and its id.

For example, the instance `Ship(id=1)` will return `U2hpcDox` as the id when you query it (which is the base64 encoding of `Ship:1`), and which could be useful later if we want to query a node by its id.

- Query "ship_node" field

In [ ]:
import base64


q = """
    query getShipNodeById($id: ID!) {
        shipNode(id: $id) {
            id
            name
        }
    }"""

id = "Ship:10"
encode_id = base64.b64encode(id.encode()).decode()

v = {"id": encode_id}
r = schema.execute(q, variables=v)
print(f'* The "r.data" of query "{q}, variables={v}\n" is: "{pr(r)}"')

The `ship_node` field in `Query` class is type of `Node.Field(Ship)`, this filed has no *resolve* method in root *Query* type, but the `get_node` method in `Ship` type should be called, and return the `Ship` object as `Node` type (with base64 encoded *id*)

- Query "node" field

In [ ]:
import base64


q = """
    query getShipNodeById($id: ID!) {
        node(id: $id) {
            id
            ... on Ship {
                name
            }
        }
    }"""

id = "Ship:10"
encode_id = base64.b64encode(id.encode()).decode()

v = {"id": encode_id}
r = schema.execute(q, variables=v)
print(f'* The "r.data" of query "{q}, variables={v}\n" is: "{pr(r)}"')

The `node` field in `Query` class is `Node` field type, and the specific type is not clear. 

By query id `"Ship:10"` ("U2hpcDoxMA==" after base64 encoded), that means query `Ship` type with physical id `10`. The `get_node` method in `Ship` type should be called.

The interface `Node` only has `id` field, so must use `... on Ship { <other fields> }` to mapping to the right type.

### 1.2. Custom Nodes

You can use the predefined `relay.Node` or you can subclass it, defining custom ways of how a node id is encoded (using the `to_global_id` method in the class) or how we can retrieve a *Node* given a encoded id (with the `get_node_from_global_id` method).

Example of custom node:

- Define dataset

In [ ]:
from promise import Promise
from promise.dataloader import DataLoader


class Dataset:
    def __init__(self):
        self.users = {}
        self.photos = {}

    def get_user(self, id):
        return self.users[id]

    def get_photo(self, id):
        return self.photos[id]

    def save_user(self, user):
        self.users[user.id] = user

    def save_photo(self, photo):
        self.photos[photo.id] = photo

    def user_count(self):
        return len(self.users)


dataset = Dataset()


class UserLoader(DataLoader):
    def batch_load_fn(self, keys):
        users = map(lambda key: dataset.get_user(key), keys)
        return Promise.resolve(list(users))


user_loader = UserLoader()


class PhotoLoader(DataLoader):
    def batch_load_fn(self, keys):
        photos = map(lambda key: dataset.get_photo(key), keys)
        return Promise.resolve(list(photos))


photo_loader = PhotoLoader()

#### 1.2.1. Custom global id

- Define CustomNode, ObjectTypes and Schema

In [ ]:
from graphene import ObjectType, String, Field, DateTime, Argument, ID, Schema
from graphene.relay import Node


class CustomNode(Node):
    class Meta:
        name = "Node"

    @staticmethod
    def to_global_id(type_, id):
        return f"{type_}:{id}"

    @staticmethod
    def get_node_from_global_id(info, global_id, only_type=None):
        type_, id = global_id.split(":", 1)
        if only_type:
            assert type_ == only_type._meta.name, (f'Type "{type_}" and '
                                                   f'"{only_type}" not match')

        # Get type from name string by schema object
        return info.schema.get_type(type_).graphene_type.get_node(info, id)


class User(ObjectType):
    class Meta:
        interfaces = (CustomNode,)

    name = String(required=True)

    @classmethod
    def get_node(cls, info, id):
        return user_loader.load(int(id))


class Photo(ObjectType):
    class Meta:
        interfaces = (CustomNode,)

    for_user = Field(User)
    datetime = DateTime()

    def resolve_for_user(self, info):
        return user_loader.load(self.for_user)

    @classmethod
    def get_node(cls, info, id):
        return dataset.get_photo(int(id))


class Query(ObjectType):
    user = CustomNode.Field(User)
    photo = CustomNode.Field(Photo)
    node = CustomNode.Field()


schema = Schema(query=Query)

The `get_node_from_global_id` method will be called when `CustomNode.Field` is resolved.

- Fill data to dataset

In [ ]:
import random
import time
from datetime import datetime


for id in range(1, 21):
    dataset.save_user(
        User(
            id=id,
            name=f"User-{id}"
        )
    )


start_date = int(time.mktime(time.strptime("2020-01-01 00:00:00", "%Y-%m-%d %H:%M:%S")))
end_date = int(time.mktime(time.strptime("2020-09-01 00:00:00", "%Y-%m-%d %H:%M:%S")))


for id in range(1, 21):
    dataset.save_photo(
        Photo(
            id=id,
            for_user=random.randint(1, dataset.user_count() - 1),
            datetime=datetime.fromtimestamp(
                random.randint(start_date, end_date)
            )
        )
    )

- Query "user" and "photo" field

In [ ]:
q = """
    query getUser($id: ID!) {
        user(id: $id) {
            id
            name
        }
    }

    query getPhoto($id: ID!) {
        photo(id: $id) {
            id
            forUser {
                id
                name
            }
            datetime
        }
    }"""

v = {"id": "User:12"}
n = "getUser"
r = schema.execute(q, operation_name=n, variables=v)
print(f'* The "r.data" of query "{q}, operation_name="{n}", variables={v}\n" '
      f'is: "{pr(r)}"')

v = {"id": "Photo:12"}
n = "getPhoto"
r = schema.execute(q, operation_name="getPhoto", variables=v)
print(f'* The "r.data" of query "{q}, operation_name="{n}", variables={v}\n" '
      f'is: "{pr(r)}"')

- Query "node" field

In [ ]:
q = """
    query getUser($id: ID!) {
        node(id: $id) {
            id
            ... on User {
                name
            }
        }
    }

    query getPhoto($id: ID!) {
        node(id: $id) {
            id
            ... on Photo {
                forUser {
                    id
                    name
                }
                datetime
            }
        }
    }"""

v = {"id": "User:12"}
n = "getUser"
r = schema.execute(q, operation_name=n, variables=v)
print(f'* The "r.data" of query "{q}, operation_name="{n}", variables={v}\n" '
      f'is: "{pr(r)}"')

v = {"id": "Photo:12"}
n = "getPhoto"
r = schema.execute(q, operation_name="getPhoto", variables=v)
print(f'* The "r.data" of query "{q}, operation_name="{n}", variables={v}\n" '
      f'is: "{pr(r)}"')

#### 1.2.2. Custom resolve node type

- Define Models

In [ ]:
class UserModel:
    def __init__(self, **kwargs):
        self.id = kwargs["id"]
        self.name = kwargs["name"]


class PhotoModel:
    def __init__(self, **kwargs):
        self.id = kwargs["id"]
        self.for_user = kwargs["for_user"]
        self.datetime = kwargs["datetime"]

- Define CustomNode, ObjectTypes and Schema

In [ ]:
from graphene import ObjectType, String, Field, DateTime, Argument, ID, Schema
from graphene.relay import Node


class CustomNode(Node):
    class Meta:
        name = "Node"

    @classmethod
    def resolve_type(cls, instance, info):
        if isinstance(instance, (User, UserModel)):
            return User
        elif isinstance(instance, (Photo, PhotoModel)):
            return Photo

        return type(instance)


class User(ObjectType):
    class Meta:
        interfaces = (CustomNode,)

    name = String(required=True)

    @classmethod
    def get_node(cls, info, id):
        return user_loader.load(int(id))


class Photo(ObjectType):
    class Meta:
        interfaces = (CustomNode,)

    for_user = Field(User)
    datetime = DateTime()

    def resolve_for_user(self, info):
        return user_loader.load(self.for_user)

    @classmethod
    def get_node(cls, info, id):
        return photo_loader.load(int(id))


class Query(ObjectType):
    user = CustomNode.Field(User)
    photo = CustomNode.Field(Photo)
    node = CustomNode.Field()


schema = Schema(query=Query)

If no `resolve_type` method in *Custom Node* class, the error `"Abstract type 'Node' must resolve to an Object type at runtime for field 'Query.node' with value <UserModel instance>, received 'None'. Either the 'Node' type should provide a 'resolve_type' function or each possible type should provide an 'is_type_of' function."` should be raised.

- Fill data to dataset

In [ ]:
import random
import time
from datetime import datetime


for id in range(1, 21):
    dataset.save_user(
        UserModel(
            id=id,
            name=f"User-{id}"
        )
    )


start_date = int(time.mktime(time.strptime("2020-01-01 00:00:00", "%Y-%m-%d %H:%M:%S")))
end_date = int(time.mktime(time.strptime("2020-09-01 00:00:00", "%Y-%m-%d %H:%M:%S")))


for id in range(1, 21):
    dataset.save_photo(
        PhotoModel(
            id=id,
            for_user=random.randint(1, dataset.user_count() - 1),
            datetime=datetime.fromtimestamp(
                random.randint(start_date, end_date)
            )
        )
    )

- Query "user" and "photo" field

In [ ]:
import base64


q = """
    query getUser($id: ID!) {
        user(id: $id) {
            id
            name
        }
    }

    query getPhoto($id: ID!) {
        photo(id: $id) {
            id
            forUser {
                id
                name
            }
            datetime
        }
    }"""

encode_id = base64.b64encode("User:12".encode()).decode()
v = {"id": encode_id}
n = "getUser"
r = schema.execute(q, operation_name=n, variables=v)
print(f'* The "r.data" of query "{q}, operation_name="{n}", variables={v}\n" '
      f'is: "{pr(r)}"')

encode_id = base64.b64encode("Photo:12".encode()).decode()
v = {"id": encode_id}
n = "getPhoto"
r = schema.execute(q, operation_name="getPhoto", variables=v)
print(f'* The "r.data" of query "{q}, operation_name="{n}", variables={v}\n" '
      f'is: "{pr(r)}"')

- Query "node" field

In [ ]:
q = """
    query getUser($id: ID!) {
        node(id: $id) {
            id
            ... on User {
                name
            }
        }
    }

    query getPhoto($id: ID!) {
        node(id: $id) {
            id
            ... on Photo {
                forUser {
                    id
                    name
                }
                datetime
            }
        }
    }"""

encode_id = base64.b64encode("User:12".encode()).decode()
v = {"id": encode_id}
n = "getUser"
r = schema.execute(q, operation_name=n, variables=v)
print(f'* The "r.data" of query "{q}, operation_name="{n}", variables={v}\n" '
      f'is: "{pr(r)}"')

encode_id = base64.b64encode("Photo:12".encode()).decode()
v = {"id": encode_id}
n = "getPhoto"
r = schema.execute(q, operation_name="getPhoto", variables=v)
print(f'* The "r.data" of query "{q}, operation_name="{n}", variables={v}\n" '
      f'is: "{pr(r)}"')

#### 1.2.3. Define type check method

Also can use `is_type_of` method in each *Object type* with `Node` interface: 

- Define Models

In [ ]:
class UserModel:
    def __init__(self, **kwargs):
        self.id = kwargs["id"]
        self.name = kwargs["name"]


class PhotoModel:
    def __init__(self, **kwargs):
        self.id = kwargs["id"]
        self.for_user = kwargs["for_user"]
        self.datetime = kwargs["datetime"]

- Define ObjectTypes and Schema

In [ ]:
from graphene import ObjectType, String, Field, DateTime, Argument, ID, Schema
from graphene.relay import Node


class User(ObjectType):
    class Meta:
        interfaces = (Node,)

    name = String(required=True)

    @classmethod
    def get_node(cls, info, id):
        return dataset.get_user(int(id))

    @classmethod
    def is_type_of(cls, root, info):
        return isinstance(root, (cls, UserModel))


class Photo(ObjectType):
    class Meta:
        interfaces = (Node,)

    for_user = Field(User)
    datetime = DateTime()

    @staticmethod
    def resolve_for_user(parent, info):
        return dataset.get_user(parent.for_user)

    @classmethod
    def get_node(cls, info, id):
        return dataset.get_photo(int(id))

    @classmethod
    def is_type_of(cls, root, info):
        return isinstance(root, (cls, PhotoModel))


class Query(ObjectType):
    user = Node.Field(User)
    photo = Node.Field(Photo)
    node = Node.Field()


schema = Schema(query=Query)

The `is_type_of` method in `ObjectType` to check the type is match when use Typeless node

- Fill data to dataset

In [ ]:
import random
import time
from datetime import datetime


for id in range(1, 21):
    dataset.save_user(
        UserModel(
            id=id,
            name=f"User-{id}"
        )
    )


start_date = int(time.mktime(time.strptime("2020-01-01 00:00:00", "%Y-%m-%d %H:%M:%S")))
end_date = int(time.mktime(time.strptime("2020-09-01 00:00:00", "%Y-%m-%d %H:%M:%S")))


for id in range(1, 21):
    dataset.save_photo(
        PhotoModel(
            id=id,
            for_user=random.randint(1, dataset.user_count() - 1),
            datetime=datetime.fromtimestamp(
                random.randint(start_date, end_date)
            )
        )
    )

- Query "user" and "photo" field

In [ ]:
import base64


q = """
    query getUser($id: ID!) {
        user(id: $id) {
            id
            name
        }
    }

    query getPhoto($id: ID!) {
        photo(id: $id) {
            id
            forUser {
                id
                name
            }
            datetime
        }
    }"""

encode_id = base64.b64encode("User:12".encode()).decode()
v = {"id": encode_id}
n = "getUser"
r = schema.execute(q, operation_name=n, variables=v)
print(f'* The "r.data" of query "{q}, operation_name="{n}", variables={v}\n" '
      f'is: "{pr(r)}"')

encode_id = base64.b64encode("Photo:12".encode()).decode()
v = {"id": encode_id}
n = "getPhoto"
r = schema.execute(q, operation_name="getPhoto", variables=v)
print(f'* The "r.data" of query "{q}, operation_name="{n}", variables={v}\n" '
      f'is: "{pr(r)}"')

- Query "node" field

In [ ]:
q = """
    query getUser($id: ID!) {
        node(id: $id) {
            id
            ... on User {
                name
            }
        }
    }

    query getPhoto($id: ID!) {
        node(id: $id) {
            id
            ... on Photo {
                forUser {
                    id
                    name
                }
                datetime
            }
        }
    }"""

encode_id = base64.b64encode("User:12".encode()).decode()
v = {"id": encode_id}
n = "getUser"
r = schema.execute(q, operation_name=n, variables=v)
print(f'* The "r.data" of query "{q}, operation_name="{n}", variables={v}\n" '
      f'is: "{pr(r)}"')

encode_id = base64.b64encode("Photo:12".encode()).decode()
v = {"id": encode_id}
n = "getPhoto"
r = schema.execute(q, operation_name="getPhoto", variables=v)
print(f'* The "r.data" of query "{q}, operation_name="{n}", variables={v}\n" '
      f'is: "{pr(r)}"')

#### 1.2.4. Use possible_types

- Define Models

In [ ]:
class UserModel:
    def __init__(self, **kwargs):
        self.id = kwargs["id"]
        self.name = kwargs["name"]


class PhotoModel:
    def __init__(self, **kwargs):
        self.id = kwargs["id"]
        self.for_user = kwargs["for_user"]
        self.datetime = kwargs["datetime"]

- Define ObjectTypes and Schema

In [ ]:
from graphene import ObjectType, String, Field, DateTime, Argument, ID, Schema
from graphene.relay import Node


class User(ObjectType):
    class Meta:
        interfaces = (Node,)
        possible_types = (User, UserModel)

    name = String(required=True)

    @classmethod
    def get_node(cls, info, id):
        return dataset.get_user(int(id))


class Photo(ObjectType):
    class Meta:
        interfaces = (Node,)
        possible_types = (Photo, PhotoModel)

    for_user = Field(User)
    datetime = DateTime()

    @staticmethod
    def resolve_for_user(parent, info):
        return dataset.get_user(parent.for_user)

    @classmethod
    def get_node(cls, info, id):
        return dataset.get_photo(int(id))


class Query(ObjectType):
    user = Node.Field(User)
    photo = Node.Field(Photo)
    node = Node.Field()


schema = Schema(query=Query)

The `is_type_of` method in `ObjectType` to check the type is match when use Typeless node

- Fill data to dataset

In [ ]:
import random
import time
from datetime import datetime


for id in range(1, 21):
    dataset.save_user(
        UserModel(
            id=id,
            name=f"User-{id}"
        )
    )


start_date = int(time.mktime(time.strptime("2020-01-01 00:00:00", "%Y-%m-%d %H:%M:%S")))
end_date = int(time.mktime(time.strptime("2020-09-01 00:00:00", "%Y-%m-%d %H:%M:%S")))


for id in range(1, 21):
    dataset.save_photo(
        PhotoModel(
            id=id,
            for_user=random.randint(1, dataset.user_count() - 1),
            datetime=datetime.fromtimestamp(
                random.randint(start_date, end_date)
            )
        )
    )

- Query "user" and "photo" field

In [ ]:
import base64


q = """
    query getUser($id: ID!) {
        user(id: $id) {
            id
            name
        }
    }

    query getPhoto($id: ID!) {
        photo(id: $id) {
            id
            forUser {
                id
                name
            }
            datetime
        }
    }"""

encode_id = base64.b64encode("User:12".encode()).decode()
v = {"id": encode_id}
n = "getUser"
r = schema.execute(q, operation_name=n, variables=v)
print(f'* The "r.data" of query "{q}, operation_name="{n}", variables={v}\n" '
      f'is: "{pr(r)}"')

encode_id = base64.b64encode("Photo:12".encode()).decode()
v = {"id": encode_id}
n = "getPhoto"
r = schema.execute(q, operation_name="getPhoto", variables=v)
print(f'* The "r.data" of query "{q}, operation_name="{n}", variables={v}\n" '
      f'is: "{pr(r)}"')

- Query "node" field

In [ ]:
q = """
    query getUser($id: ID!) {
        node(id: $id) {
            id
            ... on User {
                name
            }
        }
    }

    query getPhoto($id: ID!) {
        node(id: $id) {
            id
            ... on Photo {
                forUser {
                    id
                    name
                }
                datetime
            }
        }
    }"""

encode_id = base64.b64encode("User:12".encode()).decode()
v = {"id": encode_id}
n = "getUser"
r = schema.execute(q, operation_name=n, variables=v)
print(f'* The "r.data" of query "{q}, operation_name="{n}", variables={v}\n" '
      f'is: "{pr(r)}"')

encode_id = base64.b64encode("Photo:12".encode()).decode()
v = {"id": encode_id}
n = "getPhoto"
r = schema.execute(q, operation_name="getPhoto", variables=v)
print(f'* The "r.data" of query "{q}, operation_name="{n}", variables={v}\n" '
      f'is: "{pr(r)}"')

### 1.3. Accessing node types

If we want to retrieve node instances from a `global_id` (scalar that identifies an instance by it’s type name and id), we can simply do `Node.get_node_from_global_id(info, global_id)`.

In the case we want to restrict the instance retrieval to a specific type, we can do: `Node.get_node_from_global_id(info, global_id, only_type=Ship)`. This will raise an error if the `global_id` doesn’t correspond to a Ship type.

## 2. Connection

A connection is a vitaminized version of a List that provides ways of slicing and paginating through it. The way you create **Connection** types in `graphene` is using `relay.Connection` and `relay.ConnectionField`

### 2.1. Quick example

If we want to create a custom **Connection** on a given node, we have to suclass the `Connection` class.  
In the following example, `extra` will be an extra field in the connection, and `other` an extra field in the **Connection Edge**

- Define dataset

In [ ]:
from promise import Promise
from promise.dataloader import DataLoader


class Dataset:
    def __init__(self):
        self.heros = {}
        self.ships = {}

    def get_hero(self, id):
        return self.heros[id]

    def save_hero(self, hero):
        self.heros[hero.id] = hero

    def get_ship(self, id):
        return self.ships[id]

    def save_ship(self, ship):
        self.ships[ship.id] = ship

    def clear(self):
        self.heros = {}
        self.ships = {}


dataset = Dataset()


class HeroLoader(DataLoader):
    def batch_load_fn(self, keys):
        heros = map(lambda key: dataset.get_hero(key), keys)
        return Promise.resolve(list(heros))


hero_loader = HeroLoader()


class ShipLoader(DataLoader):
    def batch_load_fn(self, keys):
        ships = map(lambda key: dataset.get_ship(key), keys)
        return Promise.resolve(list(ships))


ship_loader = ShipLoader()

#### 2.1.1. Use `Connection`

- Define ObjectTypes and Schema

In [ ]:
from graphene import ObjectType, String, Field, Argument, ID, Schema, List, Int
from graphene.relay import Node, Connection, ConnectionField


class Ship(ObjectType):
    id = ID(required=True)
    name = String(required=True)


class ShipConnection(Connection):
    class Meta:
        node = Ship


class Hero(ObjectType):
    id = ID(required=True)
    name = String(required=True)
    own_ships = ConnectionField(ShipConnection)

    def resolve_own_ships(self, info, **kwargs):
        return ship_loader.load_many(self.own_ships)


class Query(ObjectType):
    hero = Field(Hero, id=Argument(ID, required=True))

    def resolve_hero(self, info, id):
        return hero_loader.load(int(id))


schema = Schema(query=Query)

`own_ships` field is a `ConnectionField`, in `resolve_own_ships` method, returns a `Ship` list (or `Promise` of `Ship` list), so graphene use `ArrayConnect` to resolve the `edges` and `page_info`

- Fill data to dataset

In [ ]:
import random


dataset.clear()


last_ship_id = 1

for hero_id in range(1, 21):
    ship_ids = [last_ship_id + n for n in range(random.randint(1, 3))]
    for ship_id in ship_ids:
        dataset.save_ship(
            Ship(
                id=ship_id,
                name=f'Ship-{ship_id}'
            )
        )

    dataset.save_hero(
        Hero(
            id=hero_id,
            name=f'Hero-{hero_id}',
            own_ships=ship_ids
        )
    )

    last_ship_id = ship_ids[-1]

- Query "hero" field with "own_ships" connection field

In [ ]:
import base64


q = """
    query($id: ID!) {
        hero(id: $id) {
            id
            name
            ownShips {
                pageInfo {
                    startCursor
                    endCursor
                    hasNextPage
                    hasPreviousPage
                }
                edges {
                    cursor
                    node {
                        id
                        name
                    }
                }
            }
        }
    }"""

v = {"id": 12}
r = schema.execute(q, variables=v)
print(f'* The "r.data" of query "{q}", variables={v}\n" is: "{pr(r)}"')

page_info = r.data["hero"]["ownShips"]["pageInfo"]

start_cursor = base64.b64decode(page_info["startCursor"].encode()).decode()
print(f'  the "hero.ownShips.pageInfo.startCursor" is "{start_cursor}"')

end_cursor = base64.b64decode(page_info["endCursor"].encode()).decode()
print(f'  the "hero.ownShips.pageInfo.endCursor" is "{end_cursor}"')

edges = r.data["hero"]["ownShips"]["edges"]
for n, edge in enumerate(edges):
    cursor = base64.b64decode(edge["cursor"].encode()).decode()
    print(f'  the "hero.ownShips.edges[{n}].cursor" is "{cursor}"')

#### 2.1.2. Resolve edges and page_info

- Create ObjectTypes and Schema

In [ ]:
from graphene import ObjectType, String, Field, Argument, ID, Schema, List, Int
from graphene.relay import Node, Connection, ConnectionField, PageInfo


class Ship(ObjectType):
    id = ID(required=True)
    name = String(required=True)


class QueryResult(PageInfo):
    def __init__(self, promise, start, end, count):
        self.promise = promise
        self.start = start
        self.end = end
        self.count = count

    @property
    def start_cursor(self):
        return self.start

    @property
    def end_cursor(self):
        return self.end

    @property
    def has_next_page(self):
        return self.end < self.count

    @property
    def has_previous_page(self):
        return self.start > 0

    @property
    def edges(self):
        return self.promise.get()


class ShipConnection(Connection):
    class Meta:
        node = Ship

    class Edge:
        link = String()

    total_count = Int()

    def __init__(self, query_result):
        self.query_result = query_result

    def resolve_page_info(self, info):
        return self.query_result

    def resolve_total_count(self, info):
        return self.query_result.count

    def resolve_edges(self, info):
        start = self.query_result.start
        return [
            self.Edge(
                cursor=start + n,
                node=d,
                link=f"https://myapp.com/data/{d.id}"
            ) for n, d in enumerate(self.query_result.edges)
        ]


class Hero(ObjectType):
    id = ID(required=True)
    name = String(required=True)
    own_ships = ConnectionField(ShipConnection)

    def resolve_own_ships(self, info, **kwargs):
        """
        Resolve "own_ships" field, return "ShipConnection" object
        """
        first = max(kwargs.get("first", 0), 0)
        last = min(
            kwargs.get(
                "last",
                len(self.own_ships) - 1
            ),
            len(self.own_ships) - 1
        )
        data = ship_loader.load_many(self.own_ships[first:last+1])
        result = QueryResult(data, first, last, len(self.own_ships))
        return ShipConnection(result)


class Query(ObjectType):
    hero = Field(Hero, id=Argument(ID, required=True))

    @staticmethod
    def resolve_hero(root, info, id):
        return hero_loader.load(int(id))


schema = Schema(query=Query)

- Fill data to dataset

In [ ]:
import random


dataset.clear()


last_ship_id = 1

for hero_id in range(1, 21):
    ship_ids = [last_ship_id + n for n in range(random.randint(20, 100))]
    for ship_id in ship_ids:
        dataset.save_ship(
            Ship(
                id=ship_id,
                name=f'Ship-{ship_id}'
            )
        )

    dataset.save_hero(
        Hero(
            id=hero_id,
            name=f'Hero-{hero_id}',
            own_ships=ship_ids
        )
    )

    last_ship_id = ship_ids[-1]

- Execute query

In [ ]:
q = """
    query($id: ID!, $first: Int!, $last: Int!) {
        hero(id: $id) {
            id
            name
            ownShips(first: $first, last: $last) {
                pageInfo {
                    startCursor
                    endCursor
                    hasNextPage
                    hasPreviousPage
                }
                totalCount
                edges {
                    cursor
                    link
                    node {
                        id
                        name
                    }
                }
            }
        }
    }"""

v = {"id": 12, "first": 11, "last": 15}
r = schema.execute(q, variables=v)
print(f'* The "r.data" of query "{q}", variables={v}\n" is: "{pr(r)}"')

## 3. Mutations

Most APIs don't just allow you to read data, they also allow you to write.

### 3.1. Use ClientIDMutation

In GraphQL, this is done using mutations. Just like querites, Relay puts some addtional requirements on mutations, but Graphene nicely manages that for you. All you need to do is make your mutation a subclass of `relay.ClientIDMutation`

Examples:

- Dataset

In [ ]:
class Dataset:
    def __init__(self):
        self.ships = {}
        self.factions = {}

    def clear(self):
        self.ships = {}
        self.factions = {}

    def get_ship(self, id):
        return self.ships[id]

    def get_faction(self, id):
        return self.factions[id]

    def save_ship(self, ship):
        self.ships[ship.id] = ship

    def save_faction(self, faction):
        self.factions[faction.id] = faction


dataset = Dataset()


def create_ship(cls, ship_name, faction_id):
    ship = cls(id=len(dataset.ships) + 1, name=ship_name, faction=faction_id)
    dataset.save_ship(ship)
    return ship


def create_faction(cls, faction_key, faction_name):
    faction = cls(id=faction_key, name=faction_name)
    dataset.save_faction(faction)
    return faction

- Define ObjectTypes and Mutations

In [ ]:
from graphene import ObjectType, ID, String, Field, Argument, InputObjectType
from graphene.relay import ClientIDMutation


class Ship(ObjectType):
    id = ID(required=True)
    name = String(required=True)
    faction = Field(lambda: Faction)

    def resolve_faction(self, info):
        return dataset.get_faction(self.faction)


class Faction(ObjectType):
    id = ID(required=True)
    name = String(required=True)


class Query(ObjectType):
    ship = Field(Ship, id=Argument(ID, required=True))

    def resolve_ship(self, info, id):
        return dataset.get_ship(int(id))


class ShipInput(InputObjectType):
    ship_name = String(required=True)
    faction_id = ID(required=True)


class IntroduceShipMutation(ClientIDMutation):
    class Input:
        # ship_input = ShipInput(required=True)
        ship_name = String(required=True)
        faction_id = ID(required=True)

    ship = Field(Ship)
    faction = Field(Faction)

#     @classmethod
#     def mutate_and_get_payload(cls, root, info, ship_input, client_mutation_id=None):
#         ship = create_ship(Ship, ship_input.ship_name, ship_input.faction_id)
#         faction = dataset.get_faction(ship_input.faction_id)
#         return IntroduceShipMutation(ship=ship, faction=faction)

    @classmethod
    def mutate_and_get_payload(cls, root, info, ship_name, faction_id, client_mutation_id=None):
        ship = create_ship(Ship, ship_name, faction_id)
        faction = dataset.get_faction(faction_id)
        return IntroduceShipMutation(ship=ship, faction=faction)


class Mutation(ObjectType):
    introduce_ship = IntroduceShipMutation.Field()


schema = Schema(query=Query, mutation=Mutation)

- Fill data to dataset

In [ ]:
dataset.clear()


for faction in [
    "Galactic Republic",
    "Separatist Alliance",
    "Galactic Empire",
    "Rebel Alliance",
    "New Republic"
]:
    create_faction(Faction, faction.lower().replace(" ", "_"), faction)

- Execute mutation

    The input type of `IntroduceShipMutation.Input` named `IntroduceShipMutationInput`

In [ ]:
# q = """
#     mutation($shipInput: ShipInput!) {
#         introduceShip(input: {shipInput: $shipInput}) {
#             ship {
#                 id
#                 name
#                 faction {
#                     id
#                     name
#                 }
#             }
#             faction {
#                 id
#                 name
#             }
#         }
#     }"""

q = """
    mutation($shipInput: IntroduceShipMutationInput!) {
        introduceShip(input: $shipInput) {
            ship {
                id
                name
                faction {
                    id
                    name
                }
            }
            faction {
                id
                name
            }
        }
    }"""

v = {"shipInput": {"shipName": "Millennium Falcon", "factionId": "rebel_alliance"}}
r = schema.execute(q, variables=v)
print(f'* The "r.data" of query "{q}, variables={v}\n" is: "{pr(r)}"')

- Execute query

In [ ]:
q = """
    query($id: ID!) {
        ship(id: $id) {
            id
            name
            faction {
                id
                name
            }
        }
    }"""

v = {"id": 1}
r = schema.execute(q, variables=v)
print(f'* The "r.data" of query "{q}, variables={v}\n" is: "{pr(r)}"')